In [41]:
import numpy as np
import pandas as pd
import glob
import os, sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import datetime

In [4]:
# pd.set_option('display.max_rows', None) # 전체 행 보기
pd.options.display.max_rows = 90 # 적당히
pd.options.display.max_columns = 45

In [43]:
# JS, CMPS, SSJD, SSNC. 결측치 및 이상 데이터를 포함하는 행을 제외한 Internal Sound 열
filesinJS = []
filesinCMPS = []
filesinSSJD = []
filesinSSNC = []
faulttypelist = ['JS', 'CMPS', 'SSJD', 'SSNC'] # 결함 유형
for faulttype in faulttypelist:
    os.chdir("C:\\Users\\99kit\\Desktop\\라즈베리파이제발\\Belt_"+faulttype)
    filelist = glob.glob('**/*.txt', recursive=True) # 결함 유형 폴더에서 불러온 모든 txt 파일
    len(filelist) # 3768, 3417, 3530, 3556
    num = 0
    for i in range(len(filelist)):
        currentfile = datetime.datetime(2023, int(filelist[i][:2]), int(filelist[i][3:5]), int(filelist[i][6:8]), int(filelist[i][9:11]), int(filelist[i][12:14]))
        with open(filelist[i], 'r') as file:
            data = []
            for line in file:
                values = line.strip().split(',')
                if len(values)==3:
                    data.append(values) # Internal Sound, Ambient Temmp., Target Temp. 열에 값이 없거나 보다 많은 개수의 값이 존재하는 행은 버릴 것입니다.
            globals()['A_'+currentfile.strftime("%m%d_%H%M%S")] = pd.DataFrame(data, columns=['Internal Sound', 'Ambient Temp.', 'Target Temp.']) # 동적 변수 생성
            globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Internal Sound'] = globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Internal Sound'].astype(int)
            # globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Ambient Temp.'] = globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Ambient Temp.'].astype(float) # 온도 데이터 필요할 때 살리세요.
            # globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Target Temp.'] = globals()['A_'+currentfile.strftime("%m%d_%H%M%S")]['Target Temp.'].astype(float)
            globals()['A_'+currentfile.strftime("%m%d_%H%M%S")] = pd.DataFrame(globals()['A_'+currentfile.strftime("%m%d_%H%M%S")].loc[:, 'Internal Sound']) # 온도 데이터 필요할 때 죽이세요.
            if faulttype == 'JS':
                filesinJS.append('A_'+currentfile.strftime("%m%d_%H%M%S"))
            elif faulttype == 'CMPS':
                filesinCMPS.append('A_'+currentfile.strftime("%m%d_%H%M%S"))
            elif faulttype == 'SSJD':
                filesinSSJD.append('A_'+currentfile.strftime("%m%d_%H%M%S"))
            elif faulttype == 'SSNC':
                filesinSSNC.append('A_'+currentfile.strftime("%m%d_%H%M%S"))
            num += 1
    print(f"the number of files in {faulttype}: {num}") # 3768, 3417, 3530, 3556
print(len(filesinJS)) # 3768
filesinCMPS[-1] # CMPS 마지막 동적 변수 이름
# A_0531_232959 # 동적 변수 이름

the number of files in JS: 3768
the number of files in CMPS: 3417
the number of files in SSJD: 3530
the number of files in SSNC: 3556
3768


'A_0531_215414'

In [47]:
# 모든 폴더에 대해 시작 파일부터 1초 단위로 파일이 생성되었는지 확인.
# 누락 파일명 기억해뒀다가 신호처리 후 선형 보간 예정. 다른 방법이 없다.
faulttypelist = ['JS', 'CMPS', 'SSJD', 'SSNC'] # 결함 유형
time2namelistofJS = []
time2namelistofCMPS = []
time2namelistofSSJD = []
time2namelistofSSNC = []
for faulttype in faulttypelist:
    os.chdir("C:\\Users\\99kit\\Desktop\\라즈베리파이제발\\Belt_"+faulttype)
    filelist = glob.glob('**/*.txt', recursive=True) # 결함 유형 폴더에서 불러온 모든 txt 파일
    starttime = datetime.datetime(2023, int(filelist[0][:2]), int(filelist[0][3:5]), int(filelist[0][6:8]), int(filelist[0][9:11]), int(filelist[0][12:14])) # 각 결함 유형 폴더의 첫 번째 파일 수집 시각
    endtime = datetime.datetime(2023, int(filelist[-1][:2]), int(filelist[-1][3:5]), int(filelist[-1][6:8]), int(filelist[-1][9:11]), int(filelist[-1][12:14])) # 각 결함 유형 폴더의 마지막 파일 수집 시각
    timelist = [] # 파일이 1초에 하나씩 정상적으로 생성되었을 경우를 가정하여 각 파일의 생성 시점을 저장할 것입니다.
    currenttime = starttime
    while currenttime <= endtime:
        timelist.append(currenttime) # 파일이 1초에 하나씩 정상적으로 생성되었을 경우를 가정하고 각 파일의 생성 시점을 저장합니다.
        currenttime += datetime.timedelta(seconds=1)
    if faulttype == 'JS':
        for timeitem in timelist:
            time2name = timeitem.strftime("A_%m%d_%H%M%S")
            time2namelistofJS.append(time2name)
    elif faulttype == 'CMPS':
        for timeitem in timelist:
            time2name = timeitem.strftime("A_%m%d_%H%M%S")
            time2namelistofCMPS.append(time2name)
    elif faulttype == 'SSJD':
        for timeitem in timelist:
            time2name = timeitem.strftime("A_%m%d_%H%M%S")
            time2namelistofSSJD.append(time2name)
    elif faulttype == 'SSNC':
        for timeitem in timelist:
            time2name = timeitem.strftime("A_%m%d_%H%M%S")
            time2namelistofSSNC.append(time2name)
# len(time2namelistof어쩌구) = 3952, 3616, 3707, 3637

# 결측 파일 찾기
missingfilesofJS = []
for m in time2namelistofJS:
    if m not in filesinJS:
        missingfilesofJS.append(m)

missingfilesofCMPS = []
for m in time2namelistofCMPS:
    if m not in filesinCMPS:
        missingfilesofCMPS.append(m)

missingfilesofSSJD = []
for m in time2namelistofSSJD:
    if m not in filesinSSJD:
        missingfilesofSSJD.append(m)

missingfilesofSSNC = []
for m in time2namelistofSSNC:
    if m not in filesinSSNC:
        missingfilesofSSNC.append(m)

len(missingfilesofJS) # 184. 결측 파일 정보. 184+3768=3952
# missingfilesofCMPS
# missingfilesofSSJD
# missingfilesofSSNC
#########################################################################################################################################33

184

In [133]:
# JS. 신호처리